In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Importing librares

In [2]:
import sys
sys.path.append("../src/")

In [3]:
import pandas as pd
import multiprocessing
import matplotlib.pyplot as plt

In [4]:
import seaborn as sns

In [5]:
import collections
from tqdm import tqdm
from gensim.models import word2vec

from textDataset import *
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from wordcloud import WordCloud,STOPWORDS
from scipy.stats import norm

In [6]:
cpu_count = 2*multiprocessing.cpu_count()-1
print('Number of CPUs: {}'.format(cpu_count))

Number of CPUs: 7


# Text preprocessing

In [7]:
#path2data = '../../'
path2data = '../data/news_headlines/'

text = {
        'train': TextDataset(path2data, extension='.csv', sep=',', is_train = True),
}    

In [8]:
col = 'headline_text'
text['train'].process_data(col = col, remove_stopw = True, remove_tags=False, lemmalize = True, stem = True)

In [9]:
text['train'].data.head()

,publish_date,headline_text,subject,headline_text_data,nb_words
0,20030303,unhooked brakes to blame for taiwan train disa...,news_headlines,"[unhook, brake, blame, taiwan, train, disast]",6
1,20030918,oldest prisoner in tas released citing health,news_headlines,"[oldest, prison, ta, releas, cite, health]",6
2,20030913,nine reportedly dead in portuguese plane crash,news_headlines,"[nine, reportedli, dead, portugues, plane, crash]",6
3,20031031,nurses welcome medicare rebate plan,news_headlines,"[nurs, welcom, medicar, rebat, plan]",5
4,20030930,un cuts its iraq staff,news_headlines,"[un, cut, iraq, staff]",4


In [12]:
print('Dataset lenght: {}'.format(len(text['train'].data)))
text['train'].data = text['train'].data.drop_duplicates(subset='headline_text', keep="last")
print('Dataset lenght: {}'.format(len(text['train'].data)))

Dataset lenght: 1000001
Dataset lenght: 643715


In [ ]:
text['train'].data.publish_date = pd.to_datetime(text['train'].data.publish_date.astype(str),format='%Y-%m-%d')

In [ ]:
text['train'].data.publish_date.describe()

In [ ]:
text['train'].data.publish_date.dt.year.value_counts()

# Extracting Features

In [ ]:
# Options a given year (i.e: 2012,2013) or -1 for all 15 years
year = -1

In [ ]:
if year == -1:
    sentences = { 'train': text['train'].data[col + '_data'].values}
    X_train = sentences['train']
else:
    sentences = { 'train': text['train'].data[text['train'].data.publish_date.dt.year == year][col + '_data'].values}
    X_train = sentences['train']

In [ ]:
X_train.shape

In [ ]:
X_train[0]

In [ ]:
features_extractor = 'word2Vec'
#features_extractor = 'bow'

In [ ]:
print('Features extractor: {}'.format(features_extractor))

if features_extractor == 'bow':
    
    print('Counting ocorrences of words...')

    max_gram = 2
    
    print('ngram-range: {}'.format(str((1,max_gram))))
    vectorizer = CountVectorizer(analyzer = "word",
                                 max_features = 300, ngram_range=(1,max_gram)) 
    f_train = vectorizer.fit_transform([" ".join(x) for x in tqdm(X_train)])

    #downscale weights for words using tf–idf: “Term Frequency times Inverse Document Frequency”.
    print('Words downscaling using TF-IDF...')
    
    tfidf_transformer = TfidfTransformer()
    f_train = tfidf_transformer.fit_transform(f_train)

else:
    
    # Set values for various parameters
    num_features = 300    # Word vector dimensionality                      
    min_word_count = 50   # Minimum word count                        
    num_workers = cpu_count  # Number of threads to run in parallel
    context = 10          # Context window size                                                                                    
    downsampling = 1e-3   # Downsample setting for frequent words (default value)


    print('Creating Word2Vec Model...')
    W2Vmodel = word2vec.Word2Vec(workers=num_workers, \
                size=num_features, min_count = min_word_count, \
                window = context, sample = downsampling)

    W2Vmodel.build_vocab([x for x in tqdm(X_train)])
    W2Vmodel.train([x for x in tqdm(X_train)], \
                total_examples=W2Vmodel.corpus_count, epochs=W2Vmodel.epochs)


In [ ]:
def buildWordVector(model, tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += model[word].reshape((1, size))
            count += 1.
        except KeyError: 
            continue
    return vec

In [ ]:
if features_extractor == 'word2Vec':
    f_train = np.concatenate([buildWordVector(W2Vmodel, z, num_features) for z in map(lambda x: x, X_train)])

In [ ]:
print('Shape of train features:: {}'.format(f_train.shape))

In [ ]:
# Saving train features : 

if year != -1: np.save('./f_train'+'_'+str(year)+'.npy',f_train)
else: np.save('./f_train.npy',f_train)

In [ ]:
# Loading train features : 

if year != -1: f_train = np.load('./f_train'+'_'+str(year)+'.npy')
else:          f_train = np.load('./f_train.npy')

f_train.shape

In [ ]:
from sklearn.metrics import calinski_harabaz_score

In [ ]:
from sklearn.cluster import KMeans,MiniBatchKMeans

In [ ]:
wcss = []
cali = []
kmax = 25
for i in range(2, kmax):
    kmeans = MiniBatchKMeans(n_clusters = i, init = 'k-means++',
                             n_init=1,init_size= int(1e3),
                             random_state = 42,batch_size=int(1e3))
    kmeans.fit(f_train)
    wcss.append(kmeans.inertia_)
    ypred = kmeans.predict(f_train)
    cali.append(calinski_harabaz_score(f_train,ypred))

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(range(2, kmax), cali)
plt.title('Calinski Harabaz Score')
plt.xlabel('Number of clusters')
plt.ylabel('Score')
plt.grid()


plt.subplot(1, 2, 2)
plt.plot(range(2, kmax), wcss)
plt.title('The Elbow Method (WCSS)')
plt.xlabel('Number of clusters')
plt.ylabel('Score')
plt.grid()


plt.tight_layout()

plt.show()

if year != -1: plt.savefig('./ElbowRule'+'_'+str(year)+'.png')
else:          plt.savefig('./ElbowRule.png')



Results : 

2012    4

2013    4

2014    4

Todos   4

In [ ]:
"""def elbow_rule(f_train, max_nb_cluster = 10, distortions_method='euclidean', plot=True):

    # k means determine k
    distortions = []

    for k in range(1,max_nb_cluster):
        print('Training K-means models for {} cluster/s...'.format(k))
        kmeanModel = KMeans(n_clusters=k).fit(f_train)
        kmeanModel.fit(f_train)
        if distortions_method == 'euclidean':
            distortions.append(sum(np.min(cdist(f_train, kmeanModel.cluster_centers_, 'euclidean'), \
                                          axis=1)) / f_train.shape[0])
        #elif other distortion evaluation
        
    if plot:
        # Plot the elbow
        plt.figure(figsize=(10,5))
        plt.plot(range(1,max_nb_cluster), distortions, 'bx-')
        plt.xlabel('k')
        plt.ylabel('Distortion')
        plt.title('Elbow Method')
        plt.show()"""

# Visualizing clusters

In [ ]:
kmeans = MiniBatchKMeans(n_clusters = 4, init = 'k-means++',
                         n_init=1,init_size= int(1e3),
                         random_state = 42,batch_size=int(1e3))
kmeans.fit(f_train)
ypred = kmeans.predict(f_train)

In [ ]:
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA


def plot_pca(f_train, y_train, nb_clusters = 4):

    palette = np.array(sns.color_palette("hls", nb_clusters))

    pca = PCA(n_components=3)
    result = pca.fit_transform(f_train)
    

    fig = plt.figure(figsize=(16,8))
    ax = fig.add_subplot(111, projection='3d')

    ax.scatter(result[:, 0], result[:, 1],result[:, 2],
               s=40, c=palette[ypred.astype(np.int)])

    #    plt.figure(figsize=(8, 8))     
    #    plt.scatter(result[:, 0], result[:, 1], lw=0, s=40,
    #                    c=palette[y_train.astype(np.int)])

    plt.title('Visualization PCA')
    plt.grid()
    plt.show()

In [ ]:
plot_pca(f_train, ypred)

## Words visualization using t-SNE

In [ ]:
import bokeh.plotting as bp
from sklearn.manifold import TSNE
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook



def plot_tSNE(model,n_samples = 5000):

    
    #https://www.oreilly.com/learning/an-illustrated-introduction-to-the-t-sne-algorithm

    output_notebook()
    fig = bp.figure(plot_width=700, plot_height=600, title="A map of " + str(n_samples) + " word vectors",
        tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
        x_axis_type=None, y_axis_type=None, min_border=1)


    word_vectors = [model[w] for w in model.wv.vocab.keys()][:n_samples]
    #word_vectors = [token for token in f_matrix_train][0:n_samples]
    word_centroid_map = dict(zip( W2Vmodel.wv.index2word, ypred ))



    tsne_model = TSNE(n_components=2, verbose=1, random_state=23)
    tsne_w2v = tsne_model.fit_transform(word_vectors)

    tsne_df = pd.DataFrame(tsne_w2v, columns=['x', 'y'])
    tsne_df['words'] = [k for k in model.wv.vocab.keys()][:n_samples]

    fig.scatter(x='x', y='y', source=tsne_df)
    hover = fig.select(dict(type=HoverTool))
    hover.tooltips={"word": "@words"}
    show(fig)

In [ ]:
plot_tSNE(W2Vmodel)

# Visualizing words in clusters

In [ ]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( W2Vmodel.wv.index2word, ypred ))

# For the first 5 clusters
for cluster in range(0,4):
    
    print (f"\n Cluster {cluster}")
    words = []
    for key, value in word_centroid_map.items():
        if( value == cluster ):
            words.append(key)

    print(f'{words[:10]}')

# Quantitative Cluster Analysis

In [ ]:
ypred.shape

In [ ]:
np.unique(ypred,return_counts=True)

In [ ]:
tmp =np.concatenate([f_train,ypred.reshape(-1,1)],axis=1)
tmp.shape

In [ ]:
var=[]
for cluster in range(4): 
    var.append(np.var(tmp[tmp[:,-1]==cluster]))

In [ ]:
plt.figure(figsize=(8,4))
plt.scatter(range(0, 4), var)
plt.title('Variance within cluster')
plt.xlabel('Cluster')
plt.ticklabel_format(style='plain',axis='x',useOffset=False)
plt.ylabel('Variance')
plt.grid()
plt.show()
plt.savefig('./Variance_4C.png')

# Qualitative Cluster Analysis

In [ ]:
def get_corpus(data):
    corpus = []
    for i in data:
        for j in i:
            corpus.append(j)
    return corpus    

In [ ]:
def get_wordCloud(corpus):
    
    wordCloud = WordCloud(background_color='white',
                              stopwords=STOPWORDS,
                              width=3000,
                              height=2500,
                              random_state=42
                         ).generate(str(text['train'].data['headline_text_data']))
    return wordCloud

In [ ]:
procTextCorpus = get_corpus(text['train'].data['headline_text_data'])
rawTextCorpus = get_corpus(text['train'].data['headline_text'])

### Word Cloud Visualization

In [ ]:
sns.set_style('darkgrid')
sns.set_palette('muted')
sns.set_context("notebook", font_scale=1.5,
                rc={"lines.linewidth": 2.5})

In [ ]:
procWordCloud = get_wordCloud(procTextCorpus)

In [ ]:
fig = plt.figure(figsize=(8, 8))
plt.imshow(procWordCloud)
plt.axis('off')
plt.show()

### Most frequent words

In [ ]:
n_print = 5 # change the number to analyse the # most common words

word_counter = collections.Counter(procTextCorpus)
for word, count in word_counter.most_common(n_print):
    print(word, ": ", count)

In [ ]:
fig = plt.figure(figsize=(20,5))
plt.subplot(1,2,1)

sns.distplot(text['train'].data['nb_words'],hist=True, kde=False, bins=10, fit=norm)
plt.title("Distribution of words in headline news")
plt.xlabel('Number of words in headline news')

ax = fig.add_subplot(1,2,2)
lst = word_counter.most_common(n_print)
df = pd.DataFrame(lst, columns = ['Word', 'Count'])
plt.title('Most frequent words')
df.plot(kind="barh",x='Word',y='Count', ax=ax)